# Requirement 4
## *Best-of-both-worlds algorithms with multiple products*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize

### Environment
Both stochastic environment already designed for requirement 2 and an highly non-stationary environment, where the distribution of the valuation changes **quickly** over time and the valuations are correlated together

*(Here we'll use the same class also as stochastic one, since we just need to pass the same distribution for each round)*

In [ ]:
#valuations_distr is a list long T that contains for each round the multivariate normal distribution of each product
class MultipleProductHiglyNonStationaryEnvironment:
  def __init__(self, T, N, valuations_distr):
    self.T = T # Number of buyers (rounds)
    self.N = N # Number of types of products
    self.t = 0 # Current buyer


    # Valuation of each buyer sampled from the given distribution function:
    # TxN matrix, so for each round t we have the different valuations of each product n
    self.valuations = np.array([distr.rvs() for distr in valuations_distr])

  def round(self, p_t):
    reward = np.array([])

    # Check if the buyer valuations are higher than products prices
    for valuation, price in zip(self.valuations[self.t, :], p_t):
      if valuation >= price:
        reward = np.append(reward, price)
      else:
        reward = np.append(reward, 0.)

    # Update round
    self.t = self.t + 1
    return reward

### Agent - *Multiplicative Pacing strategy for multiple product with EXP3 as regret minimizer* **with inventory constraint** 

In [2]:
class EXP3Agent:
    def __init__(self, K, learning_rate):
        self.K = K
        self.learning_rate = learning_rate
        self.weights = np.ones(K)
        self.a_t = None
        self.x_t = np.ones(K)/K
        self.N_pulls = np.zeros(K)
        self.t = 0

    def pull_arm(self):
        self.x_t = self.weights/sum(self.weights)
        self.a_t = np.random.choice(np.arange(self.K), p=self.x_t)
        return self.a_t

    def update(self, l_t):
        l_t_tilde = l_t/self.x_t[self.a_t]
        self.weights[self.a_t] *= np.exp(-self.learning_rate*l_t_tilde)
        self.N_pulls[self.a_t] += 1
        self.t += 1

In [ ]:
class MultipleProductMultiplicativePacingAgent:
    def __init__(self, prices_set, budget, T, N, eta, learning_rate):
        self.prices_set = prices_set
        self.K = len(prices_set)
        self.learning_rate = learning_rate
        self.eta=eta
        self.budget = budget
        self.T=T
        self.N = N

        #a different exp3 agent for every product
        self.exp3_list = [EXP3Agent(self.K, self.learning_rate) for _ in range(self.N)]

        self.lmbd = 1                               # Shared lambda
        self.t = 0
        self.N_pulls = np.zeros((self.N, self.K)) # Count arm pulls per product
        self.price_index = 0
        self._lambdas = []

    def pull_arm(self):
        if self.budget < 1:
            self.p_t = np.array([1.1 for _ in range(self.N)])
            return self.p_t

        chosen_prices = []
        for i in range(self.N):
            self.price_index = self.exp3_list[i].pull_arm()
            chosen_prices.append(self.prices_set[self.price_index])

        return chosen_prices
    def update(self, f_t_list):
        rho = self.budget/ (self.T - self.t + 1) + 1.5
        c_t_list= [int(f_t > 0) for f_t in f_t_list]
        c_t_tot=sum(c_t_list)

        for i in range(self.N):
          f_t=f_t_list[i]
          c_t=c_t_list[i]
          L = f_t - self.lmbd*(c_t-rho)

          # linear rescale in the feasible interval
          L_up = 1 -(1/rho)*(-rho)
          L_low = 0 -(1/rho)*(1-rho)
          rescaled_L = (L - L_low)/(L_up-L_low)
          self.exp3_list[i].update(1 - rescaled_L) # we need to maximize L

        # update lagrangian multiplier
        self.lmbd = np.clip(self.lmbd-self.eta*(rho-c_t_tot),
                            a_min=0, a_max=1/rho)

        self._lambdas.append((self.lmbd, c_t))

        #update budget
        self.budget -= c_t_tot

    def get_budget(self):
      return self.budget

    def lambdas(self):
      return self._lambdas

### Clairvoyant - **with inventory constraint**

In [ ]:
class WindowedMultipleProductHighlyNonStationaryClairvoyant:
    def __init__(self, T, B, N, prices, valuations_distr, window_size=20):
        self.T = T
        self.B = B
        self.N = N
        self.prices = np.array(prices)
        self.P = len(prices)
        self.window_size = window_size
        self.valuations_distr = valuations_distr  # list of multivariate normal distributions (len=T)

    def get_selling_probabilities(self):
        # Compute the marginal selling probabilities for each product, price, and round
        marginal_cdfs = []

        for distr in self.valuations_distr:
            means = distr.mean
            variances = np.diag(distr.cov)
            prod_cdfs = []

            for i in range(self.N):
                norm_distr = stats.norm(loc=means[i], scale=np.sqrt(variances[i]))
                cdfs = norm_distr.cdf(self.prices)
                prod_cdfs.append(cdfs)

            marginal_cdfs.append(prod_cdfs)

        marginal_cdfs = np.array(marginal_cdfs)
        selling_prob = 1 - marginal_cdfs
        return selling_prob

    def run(self):
        selling_prob = self.get_selling_probabilities() 
        expected_utility = np.zeros(self.T)
        rho = self.B / self.T + 1.5  # Per-round budget

        M = self.T // self.window_size
        for w in range(M):
            start = w * self.window_size
            end = start + self.window_size

            avg_selling = np.mean(selling_prob[start:end], axis=0) 
            c = -(avg_selling * self.prices).reshape(-1)

            A_ub = [avg_selling.reshape(-1)]
            b_ub = [rho]

            A_eq = []
            b_eq = []
            tot_vars = self.N * self.P

            for i in range(self.N):
                eq_row = np.zeros(tot_vars)
                eq_row[i * self.P:(i + 1) * self.P] = 1
                A_eq.append(eq_row)
                b_eq.append(1)

            bounds = [(0, 1)] * tot_vars

            res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub,
                                   A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

            if res.success:
                x_w = res.x.reshape(self.N, self.P)
            else:
                x_w = np.ones((self.N, self.P)) / self.P

            # Applica strategia trovata a ogni round della finestra
            for t in range(start, end):
                util_t = np.sum(x_w * self.prices * selling_prob[t])
                expected_utility[t] = util_t

        # Handle final window
        remainder = self.T % self.window_size
        if remainder > 0:
            start = M * self.window_size
            end = self.T
            avg_selling = np.mean(selling_prob[start:end], axis=0)

            c = -(avg_selling * self.prices).reshape(-1)
            A_ub = [avg_selling.reshape(-1)]
            b_ub = [rho]

            A_eq = []
            b_eq = []
            for i in range(self.N):
                eq_row = np.zeros(self.N * self.P)
                eq_row[i * self.P:(i + 1) * self.P] = 1
                A_eq.append(eq_row)
                b_eq.append(1)

            bounds = [(0, 1)] * (self.N * self.P)

            res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub,
                                   A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

            if res.success:
                x_w = res.x.reshape(self.N, self.P)
            else:
                x_w = np.ones((self.N, self.P)) / self.P

            for t in range(start, end):
                util_t = np.sum(x_w * self.prices * selling_prob[t])
                expected_utility[t] = util_t

        return expected_utility 